## Pendahuluan

Pelajaran ini akan membahas:
- Apa itu pemanggilan fungsi dan kasus penggunaannya
- Cara membuat pemanggilan fungsi menggunakan Azure OpenAI
- Cara mengintegrasikan pemanggilan fungsi ke dalam aplikasi

## Tujuan Pembelajaran

Setelah menyelesaikan pelajaran ini, Anda akan mengetahui cara dan memahami:

- Tujuan penggunaan pemanggilan fungsi
- Menyiapkan Function Call menggunakan Azure Open AI Service
- Merancang pemanggilan fungsi yang efektif untuk kasus penggunaan aplikasi Anda


## Memahami Pemanggilan Fungsi

Untuk pelajaran ini, kita ingin membangun sebuah fitur untuk startup pendidikan kita yang memungkinkan pengguna menggunakan chatbot untuk mencari kursus teknis. Kami akan merekomendasikan kursus yang sesuai dengan tingkat keahlian, peran saat ini, dan teknologi yang diminati pengguna.

Untuk menyelesaikan ini, kita akan menggunakan kombinasi dari:
 - `Azure Open AI` untuk menciptakan pengalaman chat bagi pengguna
 - `Microsoft Learn Catalog API` untuk membantu pengguna menemukan kursus berdasarkan permintaan mereka
 - `Function Calling` untuk mengambil pertanyaan pengguna dan mengirimkannya ke sebuah fungsi agar dapat melakukan permintaan API.

Untuk memulai, mari kita lihat mengapa kita ingin menggunakan pemanggilan fungsi sejak awal:

print("Pesan pada permintaan berikutnya:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # dapatkan respons baru dari GPT di mana ia bisa melihat respons fungsi


print(second_response.choices[0].message)


### Mengapa Menggunakan Function Calling

Jika kamu sudah menyelesaikan pelajaran lain di kursus ini, kamu mungkin sudah paham betapa hebatnya menggunakan Large Language Models (LLMs). Semoga kamu juga bisa melihat beberapa keterbatasannya.

Function Calling adalah fitur dari Azure Open AI Service yang dirancang untuk mengatasi keterbatasan berikut:
1) Format respons yang konsisten
2) Kemampuan menggunakan data dari sumber lain dalam aplikasi pada konteks chat

Sebelum ada function calling, respons dari LLM biasanya tidak terstruktur dan tidak konsisten. Pengembang harus menulis kode validasi yang rumit agar bisa menangani setiap variasi respons.

Pengguna tidak bisa mendapatkan jawaban seperti "Bagaimana cuaca saat ini di Stockholm?". Ini karena model hanya terbatas pada data yang digunakan saat pelatihan.

Mari kita lihat contoh di bawah ini yang menggambarkan masalah ini:

Misalkan kita ingin membuat database data siswa agar bisa merekomendasikan kursus yang tepat untuk mereka. Di bawah ini ada dua deskripsi siswa yang sangat mirip dalam data yang mereka miliki.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Kami ingin mengirimkan ini ke LLM untuk memproses data. Nantinya, ini bisa digunakan dalam aplikasi kami untuk dikirim ke API atau disimpan di database.

Mari kita buat dua prompt identik yang akan kita gunakan untuk menginstruksikan LLM tentang informasi apa saja yang ingin kami ambil:


Kami ingin mengirimkan ini ke LLM untuk menganalisis bagian-bagian yang penting bagi produk kami. Jadi kami dapat membuat dua prompt identik untuk menginstruksikan LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Setelah membuat dua prompt ini, kita akan mengirimkannya ke LLM dengan menggunakan `openai.ChatCompletion`. Kita menyimpan prompt tersebut dalam variabel `messages` dan menetapkan peran sebagai `user`. Ini dilakukan untuk meniru pesan dari pengguna yang ditulis ke chatbot.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Meskipun prompt-nya sama dan deskripsinya mirip, kita bisa mendapatkan format yang berbeda untuk properti `Grades`.

Jika kamu menjalankan sel di atas beberapa kali, formatnya bisa saja `3.7` atau `3.7 GPA`.

Ini terjadi karena LLM menerima data tidak terstruktur dalam bentuk prompt tertulis dan juga mengembalikan data yang tidak terstruktur. Kita perlu memiliki format yang terstruktur agar kita tahu apa yang diharapkan saat menyimpan atau menggunakan data ini.

Dengan menggunakan functional calling, kita bisa memastikan bahwa kita menerima data yang terstruktur. Saat menggunakan function calling, LLM sebenarnya tidak memanggil atau menjalankan fungsi apa pun. Sebagai gantinya, kita membuat sebuah struktur yang harus diikuti LLM untuk responsnya. Kemudian kita menggunakan respons yang terstruktur tersebut untuk mengetahui fungsi apa yang harus dijalankan di aplikasi kita.


![Diagram Alur Pemanggilan Fungsi](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.id.png)


### Kasus Penggunaan untuk pemanggilan fungsi

**Memanggil Alat Eksternal**
Chatbot sangat baik dalam memberikan jawaban atas pertanyaan dari pengguna. Dengan menggunakan pemanggilan fungsi, chatbot bisa memanfaatkan pesan dari pengguna untuk menyelesaikan tugas tertentu. Misalnya, seorang siswa bisa meminta chatbot untuk "Kirim email ke dosen saya bahwa saya butuh bantuan lebih lanjut dengan mata kuliah ini". Ini bisa memanggil fungsi `send_email(to: string, body: string)`

**Membuat Query API atau Database**
Pengguna bisa mencari informasi menggunakan bahasa alami yang kemudian diubah menjadi query atau permintaan API yang terformat. Contohnya, seorang guru meminta "Siapa saja siswa yang sudah menyelesaikan tugas terakhir" yang bisa memanggil fungsi bernama `get_completed(student_name: string, assignment: int, current_status: string)`

**Membuat Data Terstruktur**
Pengguna bisa mengambil sebuah blok teks atau CSV dan menggunakan LLM untuk mengekstrak informasi penting dari sana. Misalnya, seorang siswa bisa mengubah artikel Wikipedia tentang perjanjian damai untuk membuat kartu kilat AI. Ini bisa dilakukan dengan menggunakan fungsi bernama `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Membuat Panggilan Fungsi Pertama Anda

Proses membuat panggilan fungsi terdiri dari 3 langkah utama:
1. Memanggil Chat Completions API dengan daftar fungsi Anda dan pesan dari pengguna
2. Membaca respons model untuk melakukan sebuah aksi, misalnya menjalankan fungsi atau memanggil API
3. Melakukan panggilan lagi ke Chat Completions API dengan respons dari fungsi Anda untuk menggunakan informasi tersebut dalam membuat respons kepada pengguna.


![Alur Pemanggilan Fungsi](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.id.png)


### Elemen-elemen dari pemanggilan fungsi

#### Input Pengguna

Langkah pertama adalah membuat pesan dari pengguna. Ini bisa diisi secara dinamis dengan mengambil nilai dari input teks atau Anda bisa menetapkan nilainya di sini. Jika ini adalah pertama kalinya Anda bekerja dengan Chat Completions API, kita perlu mendefinisikan `role` dan `content` dari pesan tersebut.

`role` bisa berupa `system` (membuat aturan), `assistant` (model), atau `user` (pengguna akhir). Untuk pemanggilan fungsi, kita akan menetapkannya sebagai `user` dan memberikan contoh pertanyaan.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Membuat fungsi.

Selanjutnya kita akan mendefinisikan sebuah fungsi beserta parameter-parameternya. Di sini kita hanya akan menggunakan satu fungsi bernama `search_courses`, namun kamu bisa membuat beberapa fungsi.

**Penting**: Fungsi-fungsi akan dimasukkan ke dalam pesan sistem ke LLM dan akan dihitung dalam jumlah token yang tersedia untuk kamu.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definisi**

`name` - Nama fungsi yang ingin kita panggil.

`description` - Ini adalah deskripsi tentang cara kerja fungsi tersebut. Di sini penting untuk menjelaskan secara spesifik dan jelas.

`parameters` - Daftar nilai dan format yang ingin Anda minta model hasilkan dalam responsnya.

`type` - Tipe data dari properti yang akan disimpan.

`properties` - Daftar nilai spesifik yang akan digunakan model untuk responsnya.

`name` - Nama properti yang akan digunakan model dalam respons yang sudah diformat.

`type` - Tipe data dari properti ini.

`description` - Deskripsi dari properti tertentu.

**Opsional**

`required` - Properti yang wajib diisi agar pemanggilan fungsi dapat diselesaikan.


### Memanggil fungsi
Setelah mendefinisikan sebuah fungsi, sekarang kita perlu memasukkannya dalam pemanggilan ke Chat Completion API. Kita melakukan ini dengan menambahkan `functions` ke permintaan. Dalam hal ini `functions=functions`.

Ada juga opsi untuk mengatur `function_call` ke `auto`. Ini berarti kita membiarkan LLM yang memutuskan fungsi mana yang sebaiknya dipanggil berdasarkan pesan pengguna, bukan kita yang menentukannya sendiri.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Sekarang mari kita lihat responsnya dan bagaimana formatnya:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Kamu bisa melihat bahwa nama fungsi dipanggil dan dari pesan pengguna, LLM dapat menemukan data yang sesuai untuk mengisi argumen fungsi tersebut.


## 3. Mengintegrasikan Pemanggilan Fungsi ke dalam Aplikasi.

Setelah kita menguji respons yang diformat dari LLM, sekarang kita bisa mengintegrasikannya ke dalam aplikasi.

### Mengelola alur

Untuk mengintegrasikan ini ke dalam aplikasi kita, mari lakukan langkah-langkah berikut:

Pertama, lakukan pemanggilan ke layanan Open AI dan simpan pesan tersebut dalam variabel bernama `response_message`.


In [ ]:
response_message = response.choices[0].message

Sekarang kita akan mendefinisikan fungsi yang akan memanggil Microsoft Learn API untuk mendapatkan daftar kursus:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Sebagai praktik terbaik, kita akan melihat apakah model ingin memanggil sebuah fungsi. Setelah itu, kita akan membuat salah satu fungsi yang tersedia dan mencocokkannya dengan fungsi yang sedang dipanggil. 
Kemudian, kita akan mengambil argumen dari fungsi tersebut dan memetakannya ke argumen dari LLM.

Terakhir, kita akan menambahkan pesan pemanggilan fungsi dan nilai-nilai yang dikembalikan oleh pesan `search_courses`. Ini memberikan semua informasi yang dibutuhkan LLM
untuk merespons pengguna menggunakan bahasa alami.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Tantangan Kode

Kerja bagus! Untuk melanjutkan pembelajaran Anda tentang Azure Open AI Function Calling, Anda bisa membangun: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Tambahkan lebih banyak parameter pada fungsi yang bisa membantu pembelajar menemukan lebih banyak kursus. Anda bisa menemukan parameter API yang tersedia di sini:
 - Buat pemanggilan fungsi lain yang mengambil informasi tambahan dari pembelajar seperti bahasa asli mereka
 - Buat penanganan error ketika pemanggilan fungsi dan/atau pemanggilan API tidak mengembalikan kursus yang sesuai



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan terjemahan yang akurat, harap diketahui bahwa terjemahan otomatis dapat mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemah profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang timbul akibat penggunaan terjemahan ini.
